In [ ]:
!pip install ultralytics


In [ ]:
import torch
import cv2
import numpy as np
from ultralytics import YOLO

# Télécharger et extraire le dataset
!curl -L "https://universe.roboflow.com/ds/JgRQTa802s?key=ie56guH8Nm" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip


Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
 extracting: train/images/Video3_224_jpg.rf.d1f56dd3d3615eca54c63dbfc5acda6c.jpg  
 extracting: train/images/Video3_226_jpg.rf.daa12054d85d20cdfd68e80194b1af4c.jpg  
 extracting: train/images/Video3_227_jpg.rf.8e03071d671a8886d0e08a1728f37775.jpg  
 extracting: train/images/Video3_228_jpg.rf.1738369e86663975a3b9073ecc566f91.jpg  
 extracting: train/images/Video3_228_jpg.rf.787c53376a4c3bd5b4cb6577616e6650.jpg  
 extracting: train/images/Video3_229_jpg.rf.76250fc8206dd9969c75dbd28097daa9.jpg  
 extracting: train/images/Video3_229_jpg.rf.8ea4fa979e8b68ca4f4022f8c0fcaf27.jpg  
 extracting: train/images/Video3_22_jpg.rf.0fe11213af020afda158207ac445205c.jpg  
 extracting: train/images/Video3_22_jpg.rf.9e196d0098e22c8ffae95e3951e1eca4.jpg  
 extracting: train/images/Video3_231_jpg.rf.a1f2762c4b2735e2ff9908288b85af7f.jpg  
 extracting: train/images/Video3_231_jpg.rf.c3ceb69a18f8ecad30c96e45cf4fe7db.jpg  
 extracting

In [ ]:
import os

dataset_path = "/content"
print("📂 Contenu du dataset :")
print(os.listdir(dataset_path))


📂 Contenu du dataset :
['.config', 'data.yaml', 'README.dataset.txt', 'train', 'README.roboflow.txt', 'valid', 'test', 'sample_data']


In [ ]:
import shutil

# Définir les chemins
dataset_root = "/content"  # Dossier contenant le dataset téléchargé
new_dataset_path = "/content/YOLO_Dataset"

# Déplacer les images et labels vers la bonne structure
for split in ["train", "valid", "test"]:
    os.makedirs(f"{new_dataset_path}/images/{split}", exist_ok=True)
    os.makedirs(f"{new_dataset_path}/labels/{split}", exist_ok=True)

    # Déplacer les fichiers
    if os.path.exists(f"{dataset_root}/{split}/images"):
        shutil.move(f"{dataset_root}/{split}/images", f"{new_dataset_path}/images/{split}")

    if os.path.exists(f"{dataset_root}/{split}/labels"):
        shutil.move(f"{dataset_root}/{split}/labels", f"{new_dataset_path}/labels/{split}")

print("✅ Dataset réorganisé !")


✅ Dataset réorganisé !


In [ ]:
yaml_content = """path: /content/YOLO_Dataset
train: images/train
val: images/valid
test: images/test

nc: 3  # Modifier selon le nombre réel de classes
names: ['Helmet', 'Vest', 'No_PPE']
"""

with open("/content/YOLO_Dataset/data.yaml", "w") as file:
    file.write(yaml_content)

print("✅ Fichier data.yaml créé !")


✅ Fichier data.yaml créé !


In [ ]:
import glob

label_dir = "/content/YOLO_Dataset/labels/train"

for label_file in glob.glob(os.path.join(label_dir, "*.txt")):
    with open(label_file, "r") as f:
        lines = f.readlines()

    for line in lines:
        parts = line.strip().split()
        if len(parts) != 5:
            print(f"❌ Problème dans {label_file} : {line.strip()} (format incorrect)")


In [ ]:
# Installer YOLOv8 si nécessaire
!pip install ultralytics opencv-python matplotlib

# Importer YOLO
from ultralytics import YOLO

# Vérifier le GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🔍 Utilisation de : {device}")

# Charger YOLOv8 pré-entraîné
model = YOLO("yolov8n.pt")  # Modèle nano (rapide et léger)

# Entraîner le modèle
model.train(
    data="/content/YOLO_Dataset/data.yaml",
    epochs=50,
    batch=16,
    imgsz=640,
    device=device,
    patience=5,
    optimizer="Adam",
    lr0=0.001,
    weight_decay=0.0005,
    augment=True
)


🔍 Utilisation de : cpu


100%|██████████| 6.25M/6.25M [00:00<00:00, 75.8MB/s]


Ultralytics 8.3.78 🚀 Python-3.11.11 torch-2.5.1+cu124 CPU (Intel Xeon 2.20GHz)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/YOLO_Dataset/data.yaml, epochs=50, time=None, patience=5, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cpu, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=T

100%|██████████| 755k/755k [00:00<00:00, 14.5MB/s]


Overriding model.yaml nc=80 with nc=3

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

...

val: Scanning /content/YOLO_Dataset/images/valid/labels... 0 images, 509 backgrounds, 0 corrupt: 100%|██████████| 509/509 [00:00<00:00, 4355.33it/s]

val: WARNING ⚠️ No labels found in /content/YOLO_Dataset/images/valid/labels.cache. See https://docs.ultralytics.com/datasets for dataset formatting guidance.
val: New cache created: /content/YOLO_Dataset/images/valid/labels.cache
WARNING ⚠️ No labels found in /content/YOLO_Dataset/images/valid/labels.cache, training may not work correctly. See https://docs.ultralytics.com/datasets for dataset formatting guidance.


Plotting labels to runs/detect/train/labels.jpg... 
zero-size array to reduction operation maximum which has no identity
optimizer: Adam(lr=0.001, momentum=0.937) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs/detect/train
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/50         0G          0      3.914          0          0        640: 100%|██████████| 167/167 [35:59<00:00, 12.93s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 16/16 [02:44<00:00, 10.31s/it]


RuntimeError: torch.cat(): expected a non-empty list of Tensors